In [1]:
#from solar_data_retrieval import retrieval
import pandas as pd 
import numpy as np
import sys
from sys import path
path.append('..')
from solardatatools import plot_2d
import s3fs
import boto3
from solardatatools.clear_day_detection import filter_for_sparsity
from os.path import expanduser
home = expanduser('~')
with open(home + '/.aws/credentials') as f:
        lns = f.readlines()
        key = lns[3].split(' = ')[1].strip('\n')
        secret = lns[4].split(' = ')[1].strip('\n')
s3 = boto3.client('s3', aws_access_key_id=key,aws_secret_access_key=secret)
def AWS_upload(data_to_upload_df,number_of_sites, number_of_days, kind):
    #upload to aws
    bytes_to_write = data_to_upload_df.to_csv(None).encode()
    fs = s3fs.S3FileSystem(key=key, secret=secret)
    #path needs to change!
    file_path = 's3://pv.insight.misc/data_samples/s{}_d{}_{}.csv'.format(number_of_sites,number_of_days, kind)
    with fs.open(file_path, 'wb') as f:
        f.write(bytes_to_write)
        
def progress(count, total, status='', bar_length=60):
    """
    Python command line progress bar in less than 10 lines of code. · GitHub
    https://gist.github.com/vladignatyev/06860ec2040cb497f0f3
    :param count: the current count, int
    :param total: to total count, int
    :param status: a message to display
    :return:
    """
    bar_len = bar_length
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [2]:
#set variables here: 
#choose how many sites you want in your data pipeline
number_of_sites = 50
number_of_days = 50
quantile_percent = 0.95

In [7]:
#base = 's3://pv.insight.sunpower.preprocessed'
#sunpower_summary_file = pd.read_csv(base + 'summary_file.csv')
s3.download_file('pv.insight.sunpower.preprocessed','summary_file.csv','summary_file.csv')
sunpower_summary_file = pd.read_csv('summary_file.csv')

In [8]:
sunpower_summary_file_goodsparsity = sunpower_summary_file.loc[sunpower_summary_file['overall_spaersity'] < 0.3]
sunpower_summary_file_good = sunpower_summary_file_goodsparsity.loc[sunpower_summary_file_goodsparsity['overall_quality'] > 0.7]
sunpower_summary_file_good_288 = sunpower_summary_file_good.loc[sunpower_summary_file_good['time_sample'] == 288]


In [9]:
#choose site
all_sites = sunpower_summary_file_good_288.site_ID.unique()
site_IDs_selected = np.random.choice(all_sites, number_of_sites)

In [10]:
#choose sensor
index_to_download = [] #from original summary file! 
sensor_IDs_selected = []
for site_id in site_IDs_selected:
    site1 = sunpower_summary_file_good_288.loc[sunpower_summary_file_good_288['site_ID'] == site_id]
    sensor_IDs = site1.sensor_ID.unique()
    selected_sensor = np.random.choice(sensor_IDs, 1)[0]
    index_to_download.append(sunpower_summary_file.loc[(sunpower_summary_file['site_ID'] == site_id) & (sunpower_summary_file['sensor_ID'] == selected_sensor)].index.tolist()[0])
    sensor_IDs_selected.append(selected_sensor)

In [11]:
df_meta_data = pd.DataFrame(columns=['site_ID', 'sensor_ID', 'start_timestamp', 'end_timestamp', 'duration_days', 'time_sample', 'quantile_95', 'overall_spaersity', "overall_quality", "days_selected"])
counter = 1
for data_index in index_to_download:
    s3.download_file('pv.insight.sunpower.preprocessed','matrices/{0:04d}.npy'.format(data_index),'{0:04d}.npy'.format(data_index))
    power_signals_1 = np.load('{0:04d}.npy'.format(data_index))
    valid_indices = filter_for_sparsity(power_signals_1, solver='MOSEK')
    day_numbers = np.arange(power_signals_1.shape[1])
    good_day_numbers = day_numbers[valid_indices]
    list_of_selected_dates = np.random.choice(good_day_numbers, size=number_of_days, replace=False)
    power_signals = (power_signals_1 / np.quantile(power_signals_1, quantile_percent))
    power_signals_selected_days = power_signals[:,list_of_selected_dates]
    if data_index == index_to_download[0]:
        power_signals_selected_days_all = power_signals_selected_days
    else: 
        power_signals_selected_days = power_signals_selected_days
        power_signals_selected_days_all = np.concatenate([power_signals_selected_days_all, power_signals_selected_days], axis=1)
    df_meta_data = df_meta_data.append(sunpower_summary_file.iloc[data_index,:])
    df_meta_data.loc[data_index, 'days_selected'] = str(list_of_selected_dates)
    progress(counter, number_of_days, status='', bar_length=60)
    counter = counter + 1

In [12]:
df_data_input = pd.DataFrame(data=power_signals_selected_days_all[:])
upload_quantile_values = AWS_upload(df_meta_data,number_of_sites, number_of_days, "metadata")
upload_data = AWS_upload(df_data_input,number_of_sites, number_of_days, "data_input")
